In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes

import pandas as pd
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.0.0               |             py_0         606 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be 

In [ ]:
# The code was removed by Watson Studio for sharing.

First we build a GET request to FourSquare API. Here, we find coffee shop near Milan center, in a radius of 1000 meters. Here we build a request parameter and add it to url.

In [ ]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": VERSION,
    "section": "coffee",
    "near": "Milan",
    "radius": 1000,
    "limit": 100}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [ ]:
#assign response to object and get its keys
d = data.json()["response"]
d.keys()

Most of the items on dictionary are giving some general information like the name of the city, the related geo code and etc.  Groups contain the actual search result. all the coffee shop are under the venues label

In [ ]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

Now count how many coffe shop are in the bound of Milan center

In [ ]:
d["suggestedBounds"], d["totalResults"]

In [ ]:
d["geocode"]

Let's print first item to get better idea of the data structure.

In [ ]:
d["groups"][0]["items"][0]

In [ ]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]["venue"].keys()

From retrieved location we build a new pandas dataframe

In [ ]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address = location["formattedAddress"]
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address","postalcode", "lat", "lng"])
print("found %i coffee" % len(df))
df.head()

Some density based on estimator is giving a good tip where to start a new coffee shop.  There is a HeatMap to visualize all the existing coffee shops to same map.

In [ ]:
milan_center = d["geocode"]["center"]
milan_center

In [ ]:
from folium import plugins

# create map of milan using latitude and longitude values
map_milan = folium.Map(location=[milan_center["lat"], milan_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_milan)

add_markers(df)
hm_data = df[["lat", "lng"]].values.tolist()
map_milan.add_child(plugins.HeatMap(hm_data))

map_milan

## Results

Based on these results, one possibly good location for the new Coffee Shop would be in piazza fontana, at the end of a long way that connect Milan universtity and the behind of Duomo di Milano Cathedral 

In [ ]:
lat = 45.46377
lng = 9.19356
map_milan = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Our new Coffee Shop!",
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_milan)
map_milan